# Training AlexNet on Kaggle: Dogs vs. Cats

In [1]:
# import the necessary packages
# set the matplotlib backend so figures can be saved in the background
import matplotlib

# import the necessary packages
from utils.image_preprocessor import ImageToArrayPreprocessor
from utils.image_preprocessor import ResizePreprocessor
from utils.image_preprocessor import PatchPreprocessor
from utils.image_preprocessor import MeanPreprocessor
from utils.hdf5_dataset_generator import HDF5DatasetGenerator
from classifiers.alexnet import AlexNet
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
import json
import os

Using TensorFlow backend.
/home/pedro/anaconda3/envs/computer-vision/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/pedro/anaconda3/envs/computer-vision/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/pedro/anaconda3/envs/computer-vision/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1

In [2]:
# import the necessary packages
from keras.callbacks import BaseLogger
import matplotlib.pyplot as plt
import numpy as np

class TrainingMonitor(BaseLogger):
    
    def __init__(self, figPath, jsonPath=None, startAt=0):
        # store the output path for the figure, the path to the JSON
        # serialized file, and the starting epoch
        super(TrainingMonitor, self).__init__()
        self.figPath = figPath
        self.jsonPath = jsonPath
        self.startAt = startAt
        
        
    def on_train_begin(self, logs={}):
        # initialize the history dictionary
        self.H = {}
        
        # if the JSON history path exists, load the training history
        if self.jsonPath is not None:
            if os.path.exists(self.jsonPath):
                self.H = json.loads(open(self.jsonPath).read())
                
                # check to see if a starting epoch was supplied
                if self.startAt > 0:
                    # loop over the entries in the history log and
                    # trim any entries that are past the starting epoch
                    for k in self.H.keys():
                        self.H[k] = self.H[k][:self.startAt]


    def on_epoch_end(self, epoch, logs={}):
        # loop over the logs and update the loss, accuracy, etc.
        # for the entire training process
        for (k, v) in logs.items():
            l = self.H.get(k, [])
            l.append(v)
            self.H[k] = l
            
        # check to see if the training history should be serialized to file
        if self.jsonPath is not None:
            f = open(self.jsonPath, "w")
            f.write(json.dumps(self.H))
            f.close()
            
        # ensure at least two epochs have passed before plotting
        # (epoch starts at zero)
        if len(self.H["loss"]) > 1:
            # plot the training loss and accuracy
            N = np.arange(0, len(self.H["loss"]))
            plt.style.use("ggplot")
            plt.figure()
            plt.plot(N, self.H["loss"], label="train_loss")
            plt.plot(N, self.H["val_loss"], label="val_loss")
            plt.plot(N, self.H["acc"], label="train_acc")
            plt.plot(N, self.H["val_acc"], label="val_acc")
            plt.title("Training Loss and Accuracy [Epoch {}]".format(len(self.H["loss"])))
            plt.xlabel("Epoch #")
            plt.ylabel("Loss/Accuracy")
            plt.legend()
            
            # save the figure
            plt.savefig(self.figPath)
            plt.close()

In [2]:
# define the paths to the images directory
IMAGES_PATH = "./datasets/kaggle_dogs_vs_cats/train"

# since we do not have validation data or access to the testing
# labels we need to take a number of images from the training
# data and use them instead
NUM_CLASSES = 2
NUM_VAL_IMAGES = 1250 * NUM_CLASSES
NUM_TEST_IMAGES = 1250 * NUM_CLASSES

# define the path to the output training, validation, and testing
# HDF5 files
TRAIN_HDF5 = "./datasets/kaggle_dogs_vs_cats/hdf5/train.hdf5"
VAL_HDF5 = "./datasets/kaggle_dogs_vs_cats/hdf5/val.hdf5"
TEST_HDF5 = "./datasets/kaggle_dogs_vs_cats/hdf5/test.hdf5"

# path to the output model file
MODEL_PATH = "./output/kaggle_dogs_vs_cats/alexnet_dogs_vs_cats.model"

# define the path to the dataset mean
DATASET_MEAN = "./output/kaggle_dogs_vs_cats/dogs_vs_cats_mean.json"

# define the path to the output directory used for storing plots,
# classification reports, etc.
OUTPUT_PATH = "./output/kaggle_dogs_vs_cats"

In [4]:
# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15, 
                         width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15, 
                         horizontal_flip=True, fill_mode="nearest")

In [5]:
# load the RGB means for the training set
means = json.loads(open(DATASET_MEAN).read())

In [6]:
# initialize the image preprocessors
rp = ResizePreprocessor(227, 227)
pp = PatchPreprocessor(227, 227)
mp = MeanPreprocessor(means["R"], means["G"], means["B"])
iap = ImageToArrayPreprocessor()

In [7]:
# initialize the training and validation dataset generators
batch_size = 16
trainGen = HDF5DatasetGenerator(TRAIN_HDF5, batch_size, aug=aug, preprocessors=[pp, mp, iap], classes=2)
valGen = HDF5DatasetGenerator(VAL_HDF5, batch_size, preprocessors=[rp, mp, iap], classes=2)

/home/pedro/deep-learning/computer-vision/notebooks/deep-learning-for-computer-vision-with-python/utils/hdf5_dataset_generator.py:20: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  self.db = h5py.File(dbPath)


In [8]:
# initialize the optimizer
print("[INFO] compiling model...")
opt = Adam(lr=1e-3)
model = AlexNet.build(width=227, height=227, depth=3, classes=2, reg=0.0002)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

# construct the set of callbacks
path = os.path.sep.join([OUTPUT_PATH, "{}.png".format(os.getpid())])
callbacks = [TrainingMonitor(path)]

[INFO] compiling model...


In [9]:
# train the network
model.fit_generator(
    trainGen.generator(),
    steps_per_epoch=trainGen.numImages // batch_size,
    validation_data=valGen.generator(),
    validation_steps=valGen.numImages // batch_size,
    epochs=75,
    max_queue_size=batch_size * 2,
    callbacks=callbacks, verbose=1)

Epoch 1/75
1250/1250 [==============================] - 422s 337ms/step - loss: 3.6979 - acc: 0.5383 - val_loss: 3.0082 - val_acc: 0.5909
Epoch 2/75
1250/1250 [==============================] - 406s 325ms/step - loss: 2.3321 - acc: 0.5517 - val_loss: 2.2217 - val_acc: 0.5399
Epoch 3/75
1250/1250 [==============================] - 460s 368ms/step - loss: 2.2386 - acc: 0.5277 - val_loss: 1.9910 - val_acc: 0.5399
Epoch 4/75
1250/1250 [==============================] - 423s 339ms/step - loss: 2.3746 - acc: 0.5425 - val_loss: 2.1252 - val_acc: 0.5169
Epoch 5/75
1250/1250 [==============================] - 425s 340ms/step - loss: 2.6277 - acc: 0.5607 - val_loss: 2.7544 - val_acc: 0.5962
Epoch 6/75
1250/1250 [==============================] - 422s 338ms/step - loss: 2.6493 - acc: 0.5321 - val_loss: 2.4428 - val_acc: 0.5338
Epoch 7/75
1250/1250 [==============================] - 418s 335ms/step - loss: 2.5174 - acc: 0.5109 - val_loss: 6.9522 - val_acc: 0.4819
Epoch 8/75
1250/1250 [============

/home/pedro/anaconda3/envs/computer-vision/lib/python3.6/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.181148). Check your callbacks.
  % delta_t_median)


1250/1250 [==============================] - 405s 324ms/step - loss: 2.0789 - acc: 0.5357 - val_loss: 2.0290 - val_acc: 0.5334
Epoch 12/75
1250/1250 [==============================] - 403s 323ms/step - loss: 2.0194 - acc: 0.5417 - val_loss: 1.7029 - val_acc: 0.5274
Epoch 13/75
1250/1250 [==============================] - 403s 322ms/step - loss: 1.5982 - acc: 0.5609 - val_loss: 1.7997 - val_acc: 0.5596
Epoch 14/75
1250/1250 [==============================] - 402s 322ms/step - loss: 1.4116 - acc: 0.5642 - val_loss: 1.3944 - val_acc: 0.5435
Epoch 15/75
1250/1250 [==============================] - 404s 323ms/step - loss: 1.2180 - acc: 0.5835 - val_loss: 1.3941 - val_acc: 0.6155
Epoch 16/75
1250/1250 [==============================] - 402s 322ms/step - loss: 1.0950 - acc: 0.6045 - val_loss: 1.0373 - val_acc: 0.6091
Epoch 17/75
1250/1250 [==============================] - 403s 323ms/step - loss: 0.9758 - acc: 0.6341 - val_loss: 0.9731 - val_acc: 0.6695
Epoch 18/75
1250/1250 [================

1250/1250 [==============================] - 402s 321ms/step - loss: 0.5291 - acc: 0.8601 - val_loss: 0.4484 - val_acc: 0.9050
Epoch 71/75
1250/1250 [==============================] - 403s 322ms/step - loss: 0.5201 - acc: 0.8616 - val_loss: 0.4218 - val_acc: 0.9094
Epoch 72/75
1250/1250 [==============================] - 402s 322ms/step - loss: 0.5210 - acc: 0.8610 - val_loss: 0.4212 - val_acc: 0.9130
Epoch 73/75
1250/1250 [==============================] - 402s 322ms/step - loss: 0.5213 - acc: 0.8643 - val_loss: 0.4412 - val_acc: 0.9078
Epoch 74/75
1250/1250 [==============================] - 401s 321ms/step - loss: 0.5160 - acc: 0.8620 - val_loss: 0.4243 - val_acc: 0.9018
Epoch 75/75
1250/1250 [==============================] - 403s 322ms/step - loss: 0.5091 - acc: 0.8645 - val_loss: 0.4629 - val_acc: 0.8929


In [10]:
# save the model to file
print("[INFO] serializing model...")
model.save(MODEL_PATH, overwrite=True)

[INFO] serializing model...


In [ ]:
# close the HDF5 datasets
trainGen.close()
valGen.close()

# Evaluating AlexNet

In [3]:
# import the necessary packages
from utils.image_preprocessor import ImageToArrayPreprocessor
from utils.image_preprocessor import ResizePreprocessor
from utils.image_preprocessor import CropPreprocessor
from utils.image_preprocessor import MeanPreprocessor

from utils.hdf5_dataset_generator import HDF5DatasetGenerator

from utils.ranked import rank_accuracy
from keras.models import load_model
import numpy as np
import progressbar
import json

In [5]:
# load the RGB means for the training set
means = json.loads(open(DATASET_MEAN).read())

# initialize the image preprocessors
rp = ResizePreprocessor(227, 227)
mp = MeanPreprocessor(means["R"], means["G"], means["B"])
cp = CropPreprocessor(227, 227)
iap = ImageToArrayPreprocessor()

# load the pretrained network
print("[INFO] loading model...")
model = load_model(MODEL_PATH)

# initialize the testing dataset generator, then make predictions on
# the testing data
print("[INFO] predicting on test data (no crops)...")
testGen = HDF5DatasetGenerator(TEST_HDF5, 64, preprocessors=[rp, mp, iap], classes=2)
predictions = model.predict_generator(testGen.generator(), 
                                      steps=testGen.numImages // 64, max_queue_size=64 * 2)

# compute the rank-1 and rank-5 accuracies
(rank1, _) = rank_accuracy(predictions, testGen.db["labels"])
print("[INFO] rank-1: {:.2f}%".format(rank1 * 100))
testGen.close()

[INFO] loading model...
[INFO] predicting on test data (no crops)...
[INFO] rank-1: 0.00%


In [10]:
# re-initialize the testing set generator, this time excluding the
# ‘SimplePreprocessor‘
testGen = HDF5DatasetGenerator(TEST_HDF5, 64, preprocessors=[mp], classes=2)
predictions = []

# initialize the progress bar
widgets = ["Evaluating: ", progressbar.Percentage(), " ", progressbar.Bar(), " ", progressbar.ETA()]
pbar = progressbar.ProgressBar(maxval=testGen.numImages // 64, widgets=widgets).start()

Evaluating: N/A% |                                             | ETA:  --:--:--

In [12]:
# loop over a single pass of the test data
for (i, (images, labels)) in enumerate(testGen.generator(passes=1)):
    # loop over each of the individual images
    for image in images:
        # apply the crop preprocessor to the image to generate 10
        # separate crops, then convert them from images to arrays
        crops = cp.preprocess(image)
        crops = np.array([iap.preprocess(c) for c in crops], dtype="float32")
                                                                                                                                                                                                                                                                                                                    
        # make predictions on the crops and then average them
        # together to obtain the final prediction
        pred = model.predict(crops)
        predictions.append(pred.mean(axis=0))
        
    # update the progress bar
    pbar.update(i)
    
# compute the rank-1 accuracy
pbar.finish()
print("[INFO] predicting on test data (with crops)...")
(rank1, _) = rank_accuracy(predictions, testGen.db["labels"])
print("[INFO] rank-1: {:.2f}%".format(rank1 * 100))
testGen.close()

Evaluating: 100% |#############################################| Time:  0:02:04

[INFO] predicting on test data (with crops)...
[INFO] rank-1: 0.04%
